In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import string
import pandas as pd
import re
import nltk

In [ ]:
data = pd.read_csv('../input/train1/train1.csv')
test_data = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
data.fillna('unknown', inplace = True)
test_data.fillna('unknown', inplace = True)
pd.set_option('display.max_colwidth', 100)

In [ ]:
#Remove Punctuations
def remove_punct(text):
    removed_punct = ''.join([char for char in text if char not in string.punctuation])
    return removed_punct
data['nopunct_text'] = data['comment_text'].apply(lambda x : remove_punct(x))
test_data['nopunct_text'] = test_data['comment_text'].apply(lambda x : remove_punct(x))


In [ ]:
#Tokenization
def tokenize(text):
    tokenized = re.split('\s+', text)
    return tokenized
data['tokenized_text'] = data['nopunct_text'].apply(lambda x : tokenize(x))
test_data['tokenized_text'] = test_data['nopunct_text'].apply(lambda x : tokenize(x))

In [ ]:
#Remove stopwords
from nltk.corpus import stopwords
stopword = stopwords.words('english')
def remove_stopwords(text):
    no_stopwords = [word for word in text if word not in stopword]
    return no_stopwords
# print(remove_stopwords(data['tokenized_text'][1]))
data['nostop_text'] = data['tokenized_text'].apply(lambda x : remove_stopwords(x))
test_data['nostop_text'] = test_data['tokenized_text'].apply(lambda x : remove_stopwords(x))

In [ ]:
#lemmatize the text
wn = nltk.WordNetLemmatizer()
def lemmatize(text):
    lemmatized = [wn.lemmatize(word.lower()) for word in text]
    return lemmatized
data['lemmatized_text'] = data['nostop_text'].apply(lambda x : lemmatize(x))
test_data['lemmatized_text'] = test_data['nostop_text'].apply(lambda x : lemmatize(x))

In [ ]:
#Stemming the text
ps = nltk.PorterStemmer()
def stem(text):
    stemmed = [ps.stem(word) for word in text]
    return stemmed
data['stemmed_text'] = data['lemmatized_text'].apply(lambda x : stem(x))
test_data['stemmed_text'] = test_data['lemmatized_text'].apply(lambda x : stem(x))

In [ ]:
def clean(text):
    clean_text=' '.join(word for word in text)
    return clean_text
clean_text = data['stemmed_text'].apply(lambda x: clean(x))
test_clean_text = test_data['stemmed_text'].apply(lambda x: clean(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,2)).fit(clean_text)
count_train = vectorizer.transform(clean_text)
count_test = vectorizer.transform(test_clean_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer().fit(clean_text)
tfidf_train = vectorizer.transform(clean_text)
tfidf_test = vectorizer.transform(test_clean_text)

In [ ]:
import scipy.sparse as sp
count_tfidf_train = sp.hstack([count_train, tfidf_train])
count_tfidf_test = sp.hstack([count_test, tfidf_test])
count_tfidf_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
submission_df = pd.DataFrame()
submission_df['id'] = test_data['id']
for item in labels:
    log = LogisticRegression().fit(tfidf_train, data[item])
    submission_df[item] = log.predict_proba(tfidf_test)[:,1]

In [ ]:
submission_df['id'] = submission_df['id'].astype('str')

In [ ]:
submission_df.to_csv('submission.csv', index = False)

In [ ]:
# #word2vec model attempt
# from gensim.models import word2vec
# word_matrix = data['stemmed_text']
# word_matrix.shape

# # number of features the model encodes to 
# num_features = 300

# # minimum frequency of the word in the corpus
# min_word_count = 5 

# # number of workers for parallel process the model building 
# num_workers = 4

# # window size around a word
# context = 10

# print("Training model")
# model = word2vec.Word2Vec(word_matrix, workers=num_workers, size = num_features, min_count=min_word_count, window=context)


In [ ]:
# word_matrix = []
# for sentence in data['stemmed_text']:
#     word_matrix.append([word for word in sentence if word in model.wv.vocab.keys()])
# word_vectors = model.wv.syn0

In [ ]:
# import numpy as np
# missed = 0
# features = None
# feature_vector = np.zeros(word_vectors.shape[1])
# for sentence in word_matrix:
#     count = 0
#     for word in sentence:
#         feature_vector += model[word]
#         count += 1
#     if count != 0:
#         if features is None:
#             features = feature_vector/count
#         else:
#             features = np.concatenate((features, feature_vector/count))